In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import pack_vis_sol
import matplotlib.pyplot as plt
import cupy as cp
importlib.reload(kgs)
#pack_vis.plot_polygons([kgs.center_tree] + kgs.convex_breakdown)

vast
vast


<module 'kaggle_support' from '/packing/code/analysis/../core/kaggle_support.py'>

In [2]:
dat = kgs.dill_load(kgs.temp_dir + '/controller_runs/done/ga_N86_seed116_done.pickle')

In [22]:
kgs.dill_save(kgs.temp_dir + '/temp.pickle', dat.ga.ga_list[0]._cached_offspring)
import os
os.path.getsize(kgs.temp_dir + '/temp.pickle')/1e6

2.311639

In [12]:
dat.ga.ga_list[0]

GASinglePopulationOld(comment='', seed=902198532, fitness_cost=CostCompound(comment='', scaling=1.0, costs=[AreaCost(comment='', scaling=0.01), BoundaryDistanceCost(comment='', scaling=1.0, use_kernel=True), CollisionCostExactSeparation(comment='', scaling=1.0, use_lookup_table=True, lut_N_x=900, lut_N_y=900, lut_N_theta=900, lut_trim_zeros=True)]), champions=[Population(comment='', genotype=SolutionCollectionSquareSymmetric180(comment='', xyt=array([[[-2.2730463 ,  0.75852305,  4.347579  ],
        [-0.6187073 ,  2.401791  ,  1.1754313 ],
        [-1.295693  ,  1.2909782 ,  1.1986239 ],
        [-2.1638288 , -1.5394433 ,  1.9516808 ],
        [-0.9422653 ,  1.8383497 ,  1.1965293 ],
        [-2.4695492 , -2.2866325 ,  0.41237777],
        [-1.6715404 ,  0.7758521 ,  1.2230785 ],
        [-1.4378288 , -0.6954106 ,  1.2338766 ],
        [-1.7919438 , -0.262698  ,  4.2891965 ],
        [-1.1180401 , -0.19647679,  1.2269777 ],
        [-1.3977058 ,  0.22577658,  4.36857   ],
        [-0.7